In [1]:
import pandas as pd
import numpy as np

In [2]:
# df1 = pd.read_csv('./data-temp/hourly.csv', encoding='utf-8')
df2 = pd.read_csv('./data-temp/daily.csv', encoding='utf-8')
# df3 = pd.read_csv('./data-temp/monthly.csv', encoding='utf-8')
# df4 = pd.read_csv('./data-temp/yearly.csv', encoding='utf-8')

In [3]:
# df1.drop(df1.columns[[0]], axis=1, inplace=True)
df2.drop(df2.columns[[0]], axis=1, inplace=True)
# df3.drop(df3.columns[[0]], axis=1, inplace=True)
# df4.drop(df4.columns[[0]], axis=1, inplace=True)

In [4]:
df2.drop(['U','V','temp','rh','psfc'], axis='columns', inplace=True)

In [5]:
pm25 = df2['pm2.5'].values.tolist()
pm10 = df2['pm10'].values.tolist()
so2 = df2['so2'].values.tolist()
no2 = df2['no2'].values.tolist()
co = df2['co'].values.tolist()
o3 = df2['o3'].values.tolist()

In [6]:
key_list = ['pm2.5','pm10','so2','no2','co','o3']
value_list = [pm25,pm10,so2,no2,co,o3]
dict1 = dict(zip(key_list,value_list))

In [7]:
data1 = np.array([[dict1]])

```
data = pd.read_json("./data-temp/data4.json")
data1 = data.values
```

In [8]:
print((data1[0][0]).keys())

dict_keys(['pm2.5', 'pm10', 'so2', 'no2', 'co', 'o3'])


```
Idata = [
    [0, 35, 75, 115, 150, 250, 350, 500],  # PM2.5 24小时平均
    [0, 50, 150, 250, 350, 420, 500, 600],  # PM10 24小时平均
    [0, 150, 500, 650, 800],  # SO2 1小时平均
    [0, 100, 200, 700, 1200, 2340, 3090, 3840],  # NO2 1小时平均
    [0, 5, 10, 35, 60, 90, 120, 150],  # CO 1小时平均
    [0, 160, 200, 300, 400, 800, 1000, 1200]  # O3 1小时平均
]
```

In [9]:
Idata = [
        [0, 35, 75, 115, 150, 250, 350, 500],  # PM2.5 24小时平均
        [0, 50, 150, 250, 350, 420, 500, 600],  # PM10 24小时平均
        [0, 50, 150, 475, 800, 1600, 2100, 2620],  # SO2 24小时平均
        [0, 40, 80, 180, 280, 565, 750, 940],  # NO2 24小时平均
        [0, 2, 4, 14, 24, 36, 48, 60],  # CO 24小时平均
        [0, 100, 160, 215, 265, 800]  # O3 8小时滑动平均
    ]

In [10]:
qua = [0, 50, 100, 150, 200, 300, 400, 500]

In [11]:
# pri_pol = ['pm2.5','pm10','so2','no2','co','o3']

In [12]:
IAQI = list(np.zeros(len(data1[0][0]["pm2.5"])))
# Pri_pol = list(np.zeros(len(data1[0][0]["pm2.5"])))

In [13]:
Data = np.array(
        [data1[0][0]["pm2.5"], data1[0][0]["pm10"], data1[0][0]["so2"], data1[0][0]["no2"], data1[0][0]["co"],
         data1[0][0]["o3"]])
Data = list(map(list,zip(*Data)))
Data = np.array(Data)

In [14]:
Data

array([[ 73.7684    , 105.4068    ,  42.5336    ,  69.2464    ,
          1.592     ,  16.1672    ],
       [ 25.52175926,  27.57833333,   8.96888889,   2.09990741,
          0.29416667,  62.82925926],
       [ 10.34898876,  11.54505618,   4.18876404,   2.73494382,
          0.18921348,  65.88898876],
       ...,
       [  5.0925    ,   6.45544118,   2.56426471,   4.08735294,
          0.28426471,  53.04720588],
       [  3.95382445,   5.4768652 ,   1.94413793,   3.11109718,
          0.27272727,  51.31260188],
       [ 22.55527638,  26.40175879,   6.89708543,   7.51145729,
          0.42261307,  44.79      ]])

In [15]:
T_IA = 0
i = j = k = 0
for i in range(len(Idata)):
    T_data = Data[:, i]
    T_Idata = Idata[i]
    for j in range(len(T_data)):
        for k in range(1, len(T_Idata)):
            if T_Idata[k] > T_data[j]:
                break
        if k == (len(T_Idata)-1) and T_Idata[k] < T_data[j]:
            T_IA = T_Idata[k]
        else:
            T_IA = int(round((((qua[k]-qua[k-1]) / (T_Idata[k]-T_Idata[k-1])) * (
                    T_data[j]-T_Idata[k-1]) + qua[k-1]) + 0.5))
        if T_IA > IAQI[j]:
            IAQI[j] = T_IA
            # Pri_pol[j] = pri_pol[i]  # “首要污染物”
data1[0][0]["AQI"] = IAQI
# data1[0][0]["Pri_pol"] = Pri_pol

In [16]:
# data = [data1[0][0]["AQI"],data1[0][0]["Pri_pol"]]
data = data1[0][0]["AQI"]
df_temp = pd.DataFrame(data).T
# df_temp.columns = ['AQI','Pri_pol']
df_temp.columns = ['AQI']
df_temp['AQI'] = df_temp['AQI'].astype('int')

In [17]:
df2['AQI'] = df_temp['AQI']
# df2['Pri_pol'] = df_temp['Pri_pol']

In [19]:
Lv = list()
for i in df2['AQI']:
    if i > 300:
        Lv.append('六级')
    elif i > 200:
        Lv.append('五级')
    elif i > 150:
        Lv.append('四级')
    elif i > 100:
        Lv.append('三级')
    elif i > 50:
        Lv.append('二级')
    else:
        Lv.append('一级')

In [20]:
df2['Lv'] = pd.DataFrame(Lv)

In [ ]:
df2.head()

In [ ]:
df2.info()

In [23]:
df2.to_csv('AQI_daily.csv', encoding='utf-8-sig')

In [ ]:
ar = np.arange(16).reshape(2,8)
ar

In [ ]:
ar1 = ar.reshape(8,2)
ar1

In [ ]:
arr = [[1,2,3,4],[5,6,7,8],[9,10,11,12]]
arr

In [ ]:
ar2 =[]
for i in range(len(arr[0])):
    b =[]
    for j in range(len(arr)):
        b.append(arr[j][i])
    ar2.append(b)
print(ar2)

In [ ]:
ar2 = list(map(list,zip(*arr)))
print(ar2)

In [ ]:
ar2 = [ [row[i] for row in arr] for i in range(len(arr[0])) ]
print(ar2)